<div style="display: flex; background-color:rgb(214, 234, 248 );" >
<h1 style="text-align: center;margin: auto; padding: 40px; ">Produisez une étude de marché avec Python </h1>
    <p >  
      
</p>
      
  
</div>

# Notebook 1

<div>
    <img style="float: right;margin:100px 5px 5px 1px" width="200px" src="logo.png"  height=400   />
</div>
<div style="display: flex; background-color:rgb(213, 219, 219);" >
<div>    

    
* Je suis consultant Data Analyst chez La Poule qui Chante. C'est une entreprise française d’agroalimentaire. Elle souhaite se développer à l'international.

* Pour ce but mon manager , Patrick, m'a demandé de faire une analyse pour ce lancement à l'international.

* Mon objectif sera de proposer une première analyse des groupements de pays que l’on peut cibler pour exporter nos poulets. 
Pour faire mon analyse avec les critères de l’analyse PESTEL, je peux récupérer et utiliser toutes les données en open data sur le site de la FAO ainsi que les données de la FAO que mon manager m'a fourni. 

* Pour la partie analyse, il m'a demandé de
    
    * tester la classification ascendante hiérarchique, avec un dendrogramme comme visualisation.
    *utiliser la méthode des k-means, afin d’affiner l’analyse et comparer les résultats des deux méthodes de clustering.
    *d’analyser les centroïdes des classes. 
    *réaliser une ACP afin de visualiser les résultats de mon analyse, comprendre les groupes, les liens entre les variables, les liens entre les individus...
    *un notebook Jupyter ou un fichier R avec mes analyses
    *une heatmap avec les croisements entre les clusters de pays et les différentes variables
* Pendant les analyses j'ai utilisé deux notebook :
    * Un pour nettoyage des données et préparation des data propres.
    * Un pour des analyses qui ont été demandées directement par mon manager. 
</div>

</div>


<a name="top"></a>

# Sommaire :

**Partie 1**
 - <a href="#C1">Importation des libraries</a>

 - <a href="#C2">Importation des données</a>
 
**Partie 2 : Nettoyage des données**
 - <a href="#C3">Nettoyage des données</a>
 
**Partie 3 : Faire les jointures**
 - <a href="#C4">Faire les jointures</a>
 
**Partie 4 : Vérification de données manquantes dans la dataframe df_final**
 - <a href="#C5">Vérification de données manquantes dans la dataframe df_final</a>

**Partie 5 : L'Engeristrement des data propres**
- <a href="#C6">L'Engeristrement des data propres</a>



# <a name="C1">Importation des libraries</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import decomposition
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram


# <a name="C2">Importation des données</a>

In [2]:
alimentation=pd.read_csv('DisponibiliteAlimentaire_2017.csv')
population=pd.read_csv('Population_2000_2018.csv',sep=";")
matrice_commerce=pd.read_csv("matrice_commerce.csv")
pib_croissance_valuer_macro=pd.read_csv("pib_croissance_valuer_macro.csv")
stab_politc=pd.read_csv("stabilite_politique.csv")
taux_change_devise=pd.read_csv("taux_change_devise.csv")

# <a name="C3">Nettoyage des données</a>

<a href="#top">sommaire</a>

### 1. L'analyse du fichier alimentation

In [3]:
alimentation.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [4]:
alimentation.columns

Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object')

In [5]:
# On choisit les colonnes nécessaires pour nos analyses
alimentation= alimentation[['Zone','Élément', 'Produit','Année','Valeur']]
alimentation.head()

,Zone,Élément,Produit,Année,Valeur
0,Afghanistan,Production,Blé et produits,2017,4281.0
1,Afghanistan,Importations - Quantité,Blé et produits,2017,2302.0
2,Afghanistan,Variation de stock,Blé et produits,2017,-119.0
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,0.0
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,6701.0


In [6]:
# On choisi le produit la volaille
alimentation=alimentation[alimentation["Produit"]=='Viande de Volailles']
alimentation.drop(columns=["Produit","Année"],inplace=True)
alimentation.head()

,Zone,Élément,Valeur
651,Afghanistan,Production,28.0
652,Afghanistan,Importations - Quantité,29.0
653,Afghanistan,Variation de stock,0.0
654,Afghanistan,Disponibilité intérieure,57.0
655,Afghanistan,Pertes,2.0


In [7]:
# On obtient le valeur en tonnes parce que l'unité est de milliers de tonnes
alimentation["Valeur"]= alimentation["Valeur"]*1000
alimentation.head()

,Zone,Élément,Valeur
651,Afghanistan,Production,28000.0
652,Afghanistan,Importations - Quantité,29000.0
653,Afghanistan,Variation de stock,0.0
654,Afghanistan,Disponibilité intérieure,57000.0
655,Afghanistan,Pertes,2000.0


In [8]:
# On crée un pivot table 
table_alim = pd.pivot_table(alimentation, values='Valeur', index=['Zone'],
                    columns=['Élément'],aggfunc=np.mean)
table_alim=table_alim.reset_index().rename_axis(None, axis=1)
table_alim.head()

,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,NaN,NaN,NaN,5000.0,1530.0,330.0,540.0,57000.0,NaN,29000.0,55000.0,2000.0,28000.0,0.0,NaN,NaN,0.0
1,Afrique du Sud,0.0,NaN,NaN,143000.0,35690.0,9250.0,14110.0,2118000.0,63000.0,514000.0,2035000.0,83000.0,1667000.0,0.0,NaN,NaN,0.0
2,Albanie,NaN,NaN,NaN,85000.0,16360.0,6450.0,6260.0,47000.0,0.0,38000.0,47000.0,NaN,13000.0,0.0,NaN,NaN,4000.0
3,Algérie,0.0,NaN,NaN,22000.0,6380.0,1500.0,1970.0,277000.0,0.0,2000.0,264000.0,13000.0,275000.0,0.0,NaN,NaN,0.0
4,Allemagne,NaN,NaN,NaN,71000.0,19470.0,4160.0,7960.0,1739000.0,646000.0,842000.0,1609000.0,NaN,1514000.0,-38000.0,NaN,167000.0,-29000.0


In [9]:
table_alim.columns

Index(['Zone', 'Alimentation pour touristes', 'Aliments pour animaux',
       'Autres utilisations (non alimentaire)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité intérieure', 'Exportations - Quantité',
       'Importations - Quantité', 'Nourriture', 'Pertes', 'Production',
       'Résidus', 'Semences', 'Traitement', 'Variation de stock'],
      dtype='object')

In [10]:
table_alim=table_alim[['Zone','Disponibilité alimentaire en quantité (kg/personne/an)',
                      'Exportations - Quantité',
                      'Importations - Quantité', 'Production']]

In [11]:
table_alim.head()

,Zone,Disponibilité alimentaire en quantité (kg/personne/an),Exportations - Quantité,Importations - Quantité,Production
0,Afghanistan,1530.0,NaN,29000.0,28000.0
1,Afrique du Sud,35690.0,63000.0,514000.0,1667000.0
2,Albanie,16360.0,0.0,38000.0,13000.0
3,Algérie,6380.0,0.0,2000.0,275000.0
4,Allemagne,19470.0,646000.0,842000.0,1514000.0


In [12]:
# On crée une fonction pour vérifier les doublons et les nulls

def table_verif (colonne,df):
    print("shape de df",df.shape)
    print ('-'*100,sep='\n')
    print("describe de df",sep='\n')
    print(df.describe())
    print ('-'*100,sep='\n')
    
    searchdup=colonne.duplicated().sum()
    print("*On vérifie s'il y a des doublons*",searchdup,sep='\n')
    
    if searchdup==0:
        print ("Il n'y a pas de doublons",'-'*100,sep='\n')
    else:
        print("Il y a ",searchdup," doublons",'-'*100,sep='\n') 
        
    searchnull=df.isnull().sum()
    print("*On vérifie s'il y a des valeurs nulls*",''*100,searchnull, sep='\n')    

In [13]:
table_verif(table_alim["Zone"],table_alim)

shape de df (172, 5)
----------------------------------------------------------------------------------------------------
describe de df
       Disponibilité alimentaire en quantité (kg/personne/an)  \
count                                         172.000000        
mean                                        20213.372093        
std                                         15860.311134        
min                                           130.000000        
25%                                          6440.000000        
50%                                         18090.000000        
75%                                         30037.500000        
max                                         72310.000000        

       Exportations - Quantité  Importations - Quantité    Production  
count             1.350000e+02             1.700000e+02  1.680000e+02  
mean              1.321852e+05             8.952941e+04  7.251905e+05  
std               5.137844e+05             1.866700e+05  2.50

In [14]:
# On display les valeurs nulls 
table_alim[table_alim.isna().any(axis=1)].head()


,Zone,Disponibilité alimentaire en quantité (kg/personne/an),Exportations - Quantité,Importations - Quantité,Production
0,Afghanistan,1530.0,NaN,29000.0,28000.0
13,Bahamas,43170.0,NaN,24000.0,6000.0
14,Bangladesh,1500.0,NaN,0.0,249000.0
23,Burkina Faso,2270.0,NaN,0.0,46000.0
26,Cabo Verde,17620.0,NaN,12000.0,1000.0


In [15]:
# Il y a des valeurs null et on va remplacer par 0 à la fin de notre analyse parce qu'ils sont probablement zéro.
table_alim.fillna(0,inplace=True)

In [16]:
# On change les noms des colonnes
table_alim.columns=['Pays','Dispo aliment(kg/pers/an)', 'Exportations en tonnes',
       'Importations en tonnes', 'Production en tonnes']

In [17]:
table_alim.head()

,Pays,Dispo aliment(kg/pers/an),Exportations en tonnes,Importations en tonnes,Production en tonnes
0,Afghanistan,1530.0,0.0,29000.0,28000.0
1,Afrique du Sud,35690.0,63000.0,514000.0,1667000.0
2,Albanie,16360.0,0.0,38000.0,13000.0
3,Algérie,6380.0,0.0,2000.0,275000.0
4,Allemagne,19470.0,646000.0,842000.0,1514000.0


### 2. L'analyse du fichier population

In [18]:
population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [19]:
population.columns

Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object')

In [20]:
population=population[['Zone','Valeur','Année']]
population.head()

,Zone,Valeur,Année
0,Afghanistan,20779.953,2000
1,Afghanistan,21606.988,2001
2,Afghanistan,22600.770,2002
3,Afghanistan,23680.871,2003
4,Afghanistan,24726.684,2004


In [21]:
# On choisit l'année 2017
population=population[population["Année"]==2017]
population=population.drop(columns="Année")
# On multiplie la valeur de population par 1000 parce que l'unité est de milliers de personnes
population["Valeur"]=population["Valeur"]*1000
population.head()

,Zone,Valeur
17,Afghanistan,36296113.0
36,Afrique du Sud,57009756.0
55,Albanie,2884169.0
74,Algérie,41389189.0
93,Allemagne,82658409.0


In [22]:
population.columns=["Zone","Population"]
population.head()

,Zone,Population
17,Afghanistan,36296113.0
36,Afrique du Sud,57009756.0
55,Albanie,2884169.0
74,Algérie,41389189.0
93,Allemagne,82658409.0


In [23]:
table_verif(population["Zone"],population)

shape de df (236, 2)
----------------------------------------------------------------------------------------------------
describe de df
         Population
count  2.360000e+02
mean   3.198362e+07
std    1.318949e+08
min    7.930000e+02
25%    3.803032e+05
50%    5.203510e+06
75%    1.930842e+07
max    1.421022e+09
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des doublons*
0
Il n'y a pas de doublons
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des valeurs nulls*

Zone          0
Population    0
dtype: int64


In [24]:
# On change les noms des colonnes
population.columns=['Pays','Population']
population.head()

,Pays,Population
17,Afghanistan,36296113.0
36,Afrique du Sud,57009756.0
55,Albanie,2884169.0
74,Algérie,41389189.0
93,Allemagne,82658409.0


### 3. L'analyse du fichier matrice_commerce

In [25]:
# Ce fichier nous donne des informations de quantité de l'importation et l'exportation de la France
#avec les pays partenaires. 
# L'exportation est des pays et de quantité que la France export la volaille au ce pays
# L'importation est des pays et de quantité que la France import la volaille de ce pays
# Alors pour notre analyse le plus important est de pays et de quantité de l'exportation

In [26]:
matrice_commerce.head()

,Code Domaine,Domaine,Code pays déclarant (M49),Pays déclarants,Code pays partenaire (M49),Pays partenaires,Code Élément,Élément,Code Produit (CPC),Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TM,Matrices du commerce détaillées,250,France,4,Afghanistan,5910,Exportations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,244,NaN,Donnée officielle
1,TM,Matrices du commerce détaillées,250,France,4,Afghanistan,5922,Exportations - Valeur,21121,"Viande, volaille",2017,2017,1000 US$,250,NaN,Donnée officielle
2,TM,Matrices du commerce détaillées,250,France,710,Afrique du Sud,5910,Exportations - Quantité,F1061,"Viande, poulet en boîte",2017,2017,tonnes,1,NaN,Donnée officielle
3,TM,Matrices du commerce détaillées,250,France,710,Afrique du Sud,5922,Exportations - Valeur,F1061,"Viande, poulet en boîte",2017,2017,1000 US$,12,NaN,Donnée officielle
4,TM,Matrices du commerce détaillées,250,France,8,Albanie,5909,Exportations - Quantité,02151,Poulets,2017,2017,1000 têtes,10,NaN,Donnée officielle


In [27]:
matrice_commerce["Élément"].unique()

array(['Exportations - Quantité', 'Exportations - Valeur',
       'Importations - Quantité', 'Importations - Valeur'], dtype=object)

In [28]:
matrice_commerce=matrice_commerce[(matrice_commerce["Élément"]=="Exportations - Quantité")|(matrice_commerce["Élément"]=='Importations - Quantité')]
matrice_commerce.head()

,Code Domaine,Domaine,Code pays déclarant (M49),Pays déclarants,Code pays partenaire (M49),Pays partenaires,Code Élément,Élément,Code Produit (CPC),Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TM,Matrices du commerce détaillées,250,France,4,Afghanistan,5910,Exportations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,244,NaN,Donnée officielle
2,TM,Matrices du commerce détaillées,250,France,710,Afrique du Sud,5910,Exportations - Quantité,F1061,"Viande, poulet en boîte",2017,2017,tonnes,1,NaN,Donnée officielle
4,TM,Matrices du commerce détaillées,250,France,8,Albanie,5909,Exportations - Quantité,02151,Poulets,2017,2017,1000 têtes,10,NaN,Donnée officielle
5,TM,Matrices du commerce détaillées,250,France,8,Albanie,5910,Exportations - Quantité,02151,Poulets,2017,2017,tonnes,0,NaN,Donnée officielle
7,TM,Matrices du commerce détaillées,250,France,8,Albanie,5910,Exportations - Quantité,F1061,"Viande, poulet en boîte",2017,2017,tonnes,10,NaN,Donnée officielle


In [29]:
# On choisit la volaille
matrice_commerce=matrice_commerce[matrice_commerce["Produit"]=="Viande, volaille"]
matrice_commerce.head()

,Code Domaine,Domaine,Code pays déclarant (M49),Pays déclarants,Code pays partenaire (M49),Pays partenaires,Code Élément,Élément,Code Produit (CPC),Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TM,Matrices du commerce détaillées,250,France,4,Afghanistan,5910,Exportations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,244,NaN,Donnée officielle
9,TM,Matrices du commerce détaillées,250,France,8,Albanie,5910,Exportations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,78,NaN,Donnée officielle
30,TM,Matrices du commerce détaillées,250,France,276,Allemagne,5610,Importations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,34444,NaN,Donnée officielle
32,TM,Matrices du commerce détaillées,250,France,276,Allemagne,5910,Exportations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,18920,NaN,Donnée officielle
36,TM,Matrices du commerce détaillées,250,France,24,Angola,5910,Exportations - Quantité,21121,"Viande, volaille",2017,2017,tonnes,203,NaN,Donnée officielle


In [30]:
table_matrice_commerce=pd.pivot_table(matrice_commerce,values="Valeur",index=["Pays partenaires"],columns=["Élément"],aggfunc=np.mean)
table_matrice_commerce=table_matrice_commerce.reset_index().rename_axis(None, axis=1)
table_matrice_commerce.head()

,Pays partenaires,Exportations - Quantité,Importations - Quantité
0,Afghanistan,244.0,NaN
1,Albanie,78.0,NaN
2,Allemagne,18920.0,34444.0
3,Angola,203.0,NaN
4,Arabie saoudite,97510.0,NaN


In [31]:
# Pour nos analyses la quantité des exportations sont importantes et on supprime la colonne importations
table_matrice_commerce.drop(columns=["Importations - Quantité"],inplace=True)

In [32]:
# On change les noms des colonnes
table_matrice_commerce.rename(columns={"Pays partenaires": "Pays","Exportations - Quantité":"Volaille vendu par France en tonnes"},inplace=True)
table_matrice_commerce.head()

,Pays,Volaille vendu par France en tonnes
0,Afghanistan,244.0
1,Albanie,78.0
2,Allemagne,18920.0
3,Angola,203.0
4,Arabie saoudite,97510.0


In [33]:
table_verif(table_matrice_commerce["Pays"],table_matrice_commerce)

shape de df (107, 2)
----------------------------------------------------------------------------------------------------
describe de df
       Volaille vendu par France en tonnes
count                           106.000000
mean                           2828.226415
std                           10043.359766
min                               0.000000
25%                              86.000000
50%                             355.000000
75%                            1686.250000
max                           97510.000000
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des doublons*
0
Il n'y a pas de doublons
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des valeurs nulls*

Pays                                   0
Volaille vendu par France en tonnes    1
dtype: int64


In [34]:
table_matrice_commerce[pd.isnull(table_matrice_commerce["Volaille vendu par France en tonnes"])]

,Pays,Volaille vendu par France en tonnes
12,Brésil,NaN


In [35]:
# On va remplacer la valeur null par 0 parce que Brésil est l'un de pays leader de l'exportation de la volaille
# Alors probablement la France ne vends pas la volaille à ce pays
table_matrice_commerce.fillna(0,inplace=True)
table_matrice_commerce.head()

,Pays,Volaille vendu par France en tonnes
0,Afghanistan,244.0
1,Albanie,78.0
2,Allemagne,18920.0
3,Angola,203.0
4,Arabie saoudite,97510.0


### 4. L'analyse du fichier pib_croissance_valuer_macro

In [36]:
# Ce fichier nous donne des informations sur Croissance annuelle US$,Croissance annuelle US$ par habitant
# Valeur US $(PIB),Valeur US $ par habitant (PIB par habitant)

In [37]:
pib_croissance_valuer_macro.head()

,Code Domaine,Domaine,Code zone (ISO3),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,AFG,Afghanistan,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,18896.352021,X,Sources internationales sûres,NaN
1,MK,Indicateurs macro,AFG,Afghanistan,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,520.616409,X,Sources internationales sûres,NaN
2,MK,Indicateurs macro,AFG,Afghanistan,6129,Croissance annuelle US$,22008,Produit Intérieur Brut,2017,2017,%,4.865790,Fc,Donnée calculée,NaN
3,MK,Indicateurs macro,AFG,Afghanistan,61290,Croissance annuelle US$ par habitant,22008,Produit Intérieur Brut,2017,2017,%,2.227740,Fc,Donnée calculée,NaN
4,MK,Indicateurs macro,ZAF,Afrique du Sud,6110,Valeur US $,22008,Produit Intérieur Brut,2017,2017,millions,349006.689642,X,Sources internationales sûres,NaN


In [38]:
table_pib_croissance_valuer_macro=pd.pivot_table(pib_croissance_valuer_macro,values="Valeur",index=["Zone"],columns=["Élément"],aggfunc=np.mean)
table_pib_croissance_valuer_macro=table_pib_croissance_valuer_macro.reset_index().rename_axis(None, axis=1)
table_pib_croissance_valuer_macro.head()

,Zone,Croissance annuelle US$,Croissance annuelle US$ par habitant,Valeur US $,Valeur US $ par habitant
0,Afghanistan,4.865790,2.227740,1.889635e+04,520.616409
1,Afrique du Sud,17.699362,16.114973,1.791522e+05,6121.876572
2,Albanie,8.490868,9.852298,6.575225e+03,4514.204908
3,Algérie,6.287889,4.136417,1.700970e+05,4109.696001
4,Allemagne,6.441277,5.842947,3.690849e+06,44651.829102


In [39]:
# Pour notre analyse le plus important est de PIB par habitant
table_pib_croissance_valuer_macro=table_pib_croissance_valuer_macro[["Zone","Valeur US $ par habitant"]]
table_pib_croissance_valuer_macro.head()

,Zone,Valeur US $ par habitant
0,Afghanistan,520.616409
1,Afrique du Sud,6121.876572
2,Albanie,4514.204908
3,Algérie,4109.696001
4,Allemagne,44651.829102


In [40]:
# On change les noms des colonnes
table_pib_croissance_valuer_macro.columns=["Pays","Pib par habitant $"]
table_pib_croissance_valuer_macro.head()

,Pays,Pib par habitant $
0,Afghanistan,520.616409
1,Afrique du Sud,6121.876572
2,Albanie,4514.204908
3,Algérie,4109.696001
4,Allemagne,44651.829102


In [41]:
table_verif(table_pib_croissance_valuer_macro["Pays"],table_pib_croissance_valuer_macro)

shape de df (211, 2)
----------------------------------------------------------------------------------------------------
describe de df
       Pib par habitant $
count          211.000000
mean         16934.384856
std          25494.984628
min            111.048078
25%           2132.827928
50%           6376.707362
75%          20073.940387
max         171278.066389
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des doublons*
0
Il n'y a pas de doublons
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des valeurs nulls*

Pays                  0
Pib par habitant $    0
dtype: int64


### 5. L'analyse du fichier stab_politc

In [42]:
# Ce fichier nous donne des informations sur la stabilité politique des pays

In [43]:
stab_politc.head()

,Code Domaine,Domaine,Code zone (FAO),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,2,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-2.80,X,Sources internationales sûres,NaN
1,FS,Données de la sécurité alimentaire,202,Afrique du Sud,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-0.28,X,Sources internationales sûres,NaN
2,FS,Données de la sécurité alimentaire,3,Albanie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,0.38,X,Sources internationales sûres,NaN
3,FS,Données de la sécurité alimentaire,4,Algérie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-0.92,X,Sources internationales sûres,NaN
4,FS,Données de la sécurité alimentaire,79,Allemagne,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,0.59,X,Sources internationales sûres,NaN


In [44]:
table_stab_politc=pd.pivot_table(stab_politc,values="Valeur",index=["Zone"],columns=["Produit"],aggfunc=np.mean)
table_stab_politc=table_stab_politc.reset_index().rename_axis(None, axis=1)
table_stab_politc.head()

,Zone,Stabilité politique et absence de violence/terrorisme (indice)
0,Afghanistan,-2.80
1,Afrique du Sud,-0.28
2,Albanie,0.38
3,Algérie,-0.92
4,Allemagne,0.59


In [45]:
# on change les noms des colonnes
table_stab_politc.columns=["Pays","Stabilité politique"]
table_stab_politc.head()

,Pays,Stabilité politique
0,Afghanistan,-2.80
1,Afrique du Sud,-0.28
2,Albanie,0.38
3,Algérie,-0.92
4,Allemagne,0.59


In [46]:
table_verif(table_stab_politc["Pays"],table_stab_politc)

shape de df (197, 2)
----------------------------------------------------------------------------------------------------
describe de df
       Stabilité politique
count           197.000000
mean             -0.061777
std               0.994815
min              -2.940000
25%              -0.650000
50%               0.040000
75%               0.770000
max               1.920000
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des doublons*
0
Il n'y a pas de doublons
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des valeurs nulls*

Pays                   0
Stabilité politique    0
dtype: int64


### 6. L'analyse du fichier taux_change_devise

In [47]:
# Ce fichier nous donne des informations sur la valeur de monnaie de chaque pays contre US $
# Si la valeur est grand ça veut dire que la valeur de la monnaie est faible contre US $ 

In [48]:
taux_change_devise.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code de devise ISO (ISONumeric),Devise,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,PE,Taux de change - Annuels,4,Afghanistan,971,Afghani,5540,Unités de monnaie locale standardisée par USD,2017,2017,NaN,68.026904,X,Sources internationales sûres,Data from UNSD AMA
1,PE,Taux de change - Annuels,710,Afrique du Sud,710,Rand,5540,Unités de monnaie locale standardisée par USD,2017,2017,NaN,13.333781,X,Sources internationales sûres,Data from UNSD AMA
2,PE,Taux de change - Annuels,8,Albanie,8,Lek,5540,Unités de monnaie locale standardisée par USD,2017,2017,NaN,119.099667,X,Sources internationales sûres,Data from UNSD AMA
3,PE,Taux de change - Annuels,12,Algérie,12,Algerian Dinar,5540,Unités de monnaie locale standardisée par USD,2017,2017,NaN,110.973017,X,Sources internationales sûres,Data from UNSD AMA
4,PE,Taux de change - Annuels,276,Allemagne,978,Euro,5540,Unités de monnaie locale standardisée par USD,2017,2017,NaN,0.885206,X,Sources internationales sûres,Data from UNSD AMA


In [49]:
table_taux_change_devise=pd.pivot_table(taux_change_devise,values="Valeur",index=["Zone"],columns=["Produit"],aggfunc=np.mean)
table_taux_change_devise=table_taux_change_devise.reset_index().rename_axis(None, axis=1)
table_taux_change_devise.head()

,Zone,Unités de monnaie locale standardisée par USD
0,Afghanistan,68.026904
1,Afrique du Sud,13.333781
2,Albanie,119.099667
3,Algérie,110.973017
4,Allemagne,0.885206


In [50]:
# on change les noms des colonnes
table_taux_change_devise.columns=["Pays","Monnaie locale standardisée par USD"]
table_taux_change_devise.head()

,Pays,Monnaie locale standardisée par USD
0,Afghanistan,68.026904
1,Afrique du Sud,13.333781
2,Albanie,119.099667
3,Algérie,110.973017
4,Allemagne,0.885206


In [51]:
table_verif(table_taux_change_devise["Pays"],table_taux_change_devise)

shape de df (210, 2)
----------------------------------------------------------------------------------------------------
describe de df
       Monnaie locale standardisée par USD
count                           210.000000
mean                            819.998620
std                            3470.825579
min                               0.303350
25%                               1.323791
50%                               8.148834
75%                             117.736625
max                           33226.298152
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des doublons*
0
Il n'y a pas de doublons
----------------------------------------------------------------------------------------------------
*On vérifie s'il y a des valeurs nulls*

Pays                                   0
Monnaie locale standardisée par USD    0
dtype: int64


# <a name="C4">Faire les jointures</a>
<a href="#top">sommaire</a>

In [52]:
dfs = [table_alim,population,table_matrice_commerce,table_pib_croissance_valuer_macro,table_stab_politc,table_taux_change_devise]

In [53]:
import functools as ft
df_final = ft.reduce(lambda left, right: pd.merge(left, right, on='Pays',how='left'), dfs)

In [54]:
df_final.head()

,Pays,Dispo aliment(kg/pers/an),Exportations en tonnes,Importations en tonnes,Production en tonnes,Population,Volaille vendu par France en tonnes,Pib par habitant $,Stabilité politique,Monnaie locale standardisée par USD
0,Afghanistan,1530.0,0.0,29000.0,28000.0,36296113.0,244.0,520.616409,-2.80,68.026904
1,Afrique du Sud,35690.0,63000.0,514000.0,1667000.0,57009756.0,NaN,6121.876572,-0.28,13.333781
2,Albanie,16360.0,0.0,38000.0,13000.0,2884169.0,78.0,4514.204908,0.38,119.099667
3,Algérie,6380.0,0.0,2000.0,275000.0,41389189.0,NaN,4109.696001,-0.92,110.973017
4,Allemagne,19470.0,646000.0,842000.0,1514000.0,82658409.0,18920.0,44651.829102,0.59,0.885206


# <a name="C5">Vérification de données manquantes dans la dataframe df_final</a>

<a href="#top">sommaire</a>

In [55]:
df_final.isnull().sum()

Pays                                    0
Dispo aliment(kg/pers/an)               0
Exportations en tonnes                  0
Importations en tonnes                  0
Production en tonnes                    0
Population                              0
Volaille vendu par France en tonnes    73
Pib par habitant $                      1
Stabilité politique                     3
Monnaie locale standardisée par USD     1
dtype: int64

In [56]:
df_final[df_final["Stabilité politique"].isnull()]

,Pays,Dispo aliment(kg/pers/an),Exportations en tonnes,Importations en tonnes,Production en tonnes,Population,Volaille vendu par France en tonnes,Pib par habitant $,Stabilité politique,Monnaie locale standardisée par USD
34,"Chine, continentale",12330.0,576000.0,452000.0,18236000.0,1.421022e+09,634.0,8663.126784,NaN,6.758755
108,Nouvelle-Calédonie,38710.0,0.0,9000.0,1000.0,2.771500e+05,1131.0,33101.384336,NaN,105.633078
120,Polynésie française,47400.0,0.0,15000.0,1000.0,2.761020e+05,333.0,21127.527814,NaN,105.633078


In [57]:
# On impute les valeurs nulls par les valeurs qu'on trouve sur le site FAO
df_final.loc[34,"Stabilité politique"]= -0.23
df_final.loc[108,"Stabilité politique"]= 0.28
df_final.loc[120,"Stabilité politique"]= 0.28

In [58]:
df_final[df_final["Monnaie locale standardisée par USD"].isnull()]

,Pays,Dispo aliment(kg/pers/an),Exportations en tonnes,Importations en tonnes,Production en tonnes,Population,Volaille vendu par France en tonnes,Pib par habitant $,Stabilité politique,Monnaie locale standardisée par USD
33,"Chine, Taiwan Province de",33170.0,8000.0,161000.0,652000.0,23674546.0,NaN,NaN,0.86,NaN


In [59]:
# On impute les valeurs nulls par les valeurs qu'on trouve sur internet
# source https://www.poundsterlinglive.com/history/USD-TWD-2017 
df_final.loc[33,"Monnaie locale standardisée par USD"]= 29.617

In [60]:
df_final[df_final["Pib par habitant $"].isnull()]

,Pays,Dispo aliment(kg/pers/an),Exportations en tonnes,Importations en tonnes,Production en tonnes,Population,Volaille vendu par France en tonnes,Pib par habitant $,Stabilité politique,Monnaie locale standardisée par USD
33,"Chine, Taiwan Province de",33170.0,8000.0,161000.0,652000.0,23674546.0,NaN,NaN,0.86,29.617


In [61]:
# On impute les valeurs nulls par les valeurs qu'on trouve sur internet
# source https://fr.statista.com/statistiques/1310481/taiwan-pib-habitant/
df_final.loc[33,"Pib par habitant $"]= 25.062

In [62]:
df_final.isnull().sum()

Pays                                    0
Dispo aliment(kg/pers/an)               0
Exportations en tonnes                  0
Importations en tonnes                  0
Production en tonnes                    0
Population                              0
Volaille vendu par France en tonnes    73
Pib par habitant $                      0
Stabilité politique                     0
Monnaie locale standardisée par USD     0
dtype: int64

In [63]:
# On remplace les autres valeurs nulls par zero pour la colonne Volaille vendu par France en tonnes 
df_final.fillna(0,inplace=True)

In [64]:
df_final.isnull().sum()

Pays                                   0
Dispo aliment(kg/pers/an)              0
Exportations en tonnes                 0
Importations en tonnes                 0
Production en tonnes                   0
Population                             0
Volaille vendu par France en tonnes    0
Pib par habitant $                     0
Stabilité politique                    0
Monnaie locale standardisée par USD    0
dtype: int64

In [65]:
df_final.head()

,Pays,Dispo aliment(kg/pers/an),Exportations en tonnes,Importations en tonnes,Production en tonnes,Population,Volaille vendu par France en tonnes,Pib par habitant $,Stabilité politique,Monnaie locale standardisée par USD
0,Afghanistan,1530.0,0.0,29000.0,28000.0,36296113.0,244.0,520.616409,-2.80,68.026904
1,Afrique du Sud,35690.0,63000.0,514000.0,1667000.0,57009756.0,0.0,6121.876572,-0.28,13.333781
2,Albanie,16360.0,0.0,38000.0,13000.0,2884169.0,78.0,4514.204908,0.38,119.099667
3,Algérie,6380.0,0.0,2000.0,275000.0,41389189.0,0.0,4109.696001,-0.92,110.973017
4,Allemagne,19470.0,646000.0,842000.0,1514000.0,82658409.0,18920.0,44651.829102,0.59,0.885206


# <a name="C6"> L'Engeristrement des data propres</a>

<a href="#top">sommaire</a>

In [66]:
#df_final.to_csv('/Users/Desktop/fao/df_final.csv', index=False)
